In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sea
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import xgboost as xgb
import tensorflow as tf


In [3]:
df = pd.read_csv('SOLUSTDAtas_tratado.csv')
df.head()

,date,open,high,low,close
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.4+ MB
None


# Preprocesado de Datos

In [5]:
tamanio = df.shape[0]

In [6]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_train

,date,open,high,low,close
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25
...,...,...,...,...,...
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95
25479,2023-07-12 00:00:00,21.95,22.10,21.90,22.05


In [7]:
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_vali

,date,open,high,low,close
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95
25485,2023-07-12 06:00:00,21.95,22.05,21.90,22.00
...,...,...,...,...,...
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30
32759,2024-05-10 08:00:00,153.30,155.10,153.15,154.95


In [8]:
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]
df_test

,date,open,high,low,close
32761,2024-05-10 10:00:00,154.35,154.5,153.45,154.10
32762,2024-05-10 11:00:00,154.10,154.8,153.25,154.15
32763,2024-05-10 12:00:00,154.15,154.3,153.25,154.15
32764,2024-05-10 13:00:00,154.15,155.2,153.00,155.05
32765,2024-05-10 14:00:00,155.05,155.4,153.10,153.30
...,...,...,...,...,...
36395,2024-10-08 20:00:00,143.35,143.9,142.35,142.95
36396,2024-10-08 21:00:00,142.95,144.1,142.25,143.75
36397,2024-10-08 22:00:00,143.75,144.5,143.35,144.50
36398,2024-10-08 23:00:00,144.50,144.7,144.05,144.25


# Declaramos y Entrenamos el modelo

Preparamos los datos en el formato requerido por el algoritmo para su entrenamiento

In [11]:
X1 = []
y1= []
for i in range(0, df_train.shape[0] - 1):
    auxy = df_train.iloc[i + 1]
    y1.append(auxy.close)    #Añadimos a y el valor de close de la fila i+1 de df.
    aux = df_train.iloc[i]
    aux1 = []
    for r in range(1, aux.size):  #Añadimos a aux1 los valores open, high, low y close de la fila i de df.
        aux1.append(aux[r])
    X1.append(aux1)

C:\Users\raulg\AppData\Local\Temp\ipykernel_572\744263331.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


Preparamos los datos usados para testear el algortimo en el formato requerido

In [22]:
Xvt = []
yvt = []
for n in range (0, df_vali.shape[0]):
    aux = df_vali.iloc[n]
    aux1 = aux[1:].tolist()
    Xvt.append(aux1)
for r in range (0,df_test.shape[0] - 1):
    aux = df_test.iloc[r]
    aux1 = aux[1:].tolist()
    Xvt.append(aux1)
for t in range(1, df_vali.shape[0]):
    yvt.append(df_vali.iloc[t].close)
for d in range(0, df_test.shape[0]):
    yvt.append(df_test.iloc[d].close)

In [23]:
print(Xvt[1])

[22.15, 22.25, 22.1, 22.1]


In [24]:
print(yvt[0])

22.1


Desarrollamos un algoritmo para entrenar distintas instacias con distintos parámetros y compararlas

In [25]:
def evalRandomForest(Testrpr, predictT):
    suma = 0
    n = len(Testrpr)
    for i in range(0,n):
        suma = abs(predictT[i] - Testrpr[i])/Testrpr[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [13]:
def train_randomForestdepth(ini, fin):
    posbest = 0
    best = 100
    for i in range(ini, fin):
        regr1 = RandomForestRegressor(max_depth=i, random_state=0)
        regr1.fit(X1, y1)
        predictT = regr1.predict(Xvt)
        valor = evalRandomForest(yvt, predictT)
        if valor < best:
            best = valor
            posbest = i
    return(posbest, best)
        

In [28]:
tuple = train_randomForestdepth(1, 50)

In [29]:
print(tuple[0])
print(tuple[1])

9
0.7791437185511273


# Predicción Utilizando XGBoost

Ponemos los datos en el formato correcto

In [9]:
df_xgb = df.copy()
df_xgb['close_next'] = df_xgb['close'].shift(-1)


In [10]:
df_xgb = df_xgb.dropna()
df_xgb

,date,open,high,low,close,close_next
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,3.15
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,3.30
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,3.25
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,3.25
...,...,...,...,...,...,...
36394,2024-10-08 19:00:00,143.80,143.95,142.40,143.35,142.95
36395,2024-10-08 20:00:00,143.35,143.90,142.35,142.95,143.75
36396,2024-10-08 21:00:00,142.95,144.10,142.25,143.75,144.50
36397,2024-10-08 22:00:00,143.75,144.50,143.35,144.50,144.25


In [11]:
tamanio_xgb = df_xgb.shape[0]
tamanio_xgb

36399

In [12]:
df_train_xgb = df_xgb.copy().loc[0:int(tamanio_xgb*0.7)]
df_train_xgb

,date,open,high,low,close,close_next
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,3.15
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,3.30
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,3.25
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,3.25
...,...,...,...,...,...,...
25475,2023-07-11 20:00:00,22.10,22.10,21.95,22.00,22.00
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00,22.00
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00,21.95
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95,22.05


In [13]:
df_vali_xgb = df_xgb.copy().loc[int(tamanio_xgb*0.7 + 1):int(tamanio_xgb*0.9)]
df_vali_xgb

,date,open,high,low,close,close_next
25480,2023-07-12 01:00:00,22.05,22.15,22.00,22.10,22.15
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15,22.10
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10,22.00
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00,21.95
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95,22.00
...,...,...,...,...,...,...
32755,2024-05-10 04:00:00,153.40,153.80,152.70,153.65,153.95
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95,153.75
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75,153.30
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30,154.95


In [14]:
df_test_xgb = df_xgb.copy().loc[int(tamanio_xgb*0.9 + 1):tamanio_xgb]
df_test_xgb

,date,open,high,low,close,close_next
32760,2024-05-10 09:00:00,154.95,155.75,154.25,154.35,154.10
32761,2024-05-10 10:00:00,154.35,154.50,153.45,154.10,154.15
32762,2024-05-10 11:00:00,154.10,154.80,153.25,154.15,154.15
32763,2024-05-10 12:00:00,154.15,154.30,153.25,154.15,155.05
32764,2024-05-10 13:00:00,154.15,155.20,153.00,155.05,153.30
...,...,...,...,...,...,...
36394,2024-10-08 19:00:00,143.80,143.95,142.40,143.35,142.95
36395,2024-10-08 20:00:00,143.35,143.90,142.35,142.95,143.75
36396,2024-10-08 21:00:00,142.95,144.10,142.25,143.75,144.50
36397,2024-10-08 22:00:00,143.75,144.50,143.35,144.50,144.25


In [15]:
df_valitest_xgb = pd.concat([df_vali_xgb, df_test_xgb], ignore_index=True)

In [16]:
XvtXGB = df_valitest_xgb[['open', 'high', 'low', 'close']] 
yvtXGB = df_valitest_xgb['close_next']

In [17]:
dtrainvali = xgb.DMatrix(data=XvtXGB, label=yvtXGB)

Definimos las características para el entrenamiento

In [20]:
X = df_train_xgb[['open', 'high', 'low', 'close']] 
y = df_train_xgb['close_next']
X1 = df_vali_xgb[['open', 'high', 'low', 'close']] 
y1 = df_vali_xgb['close_next']

In [21]:
dtrain = xgb.DMatrix(data=X, label=y)
dvali = xgb.DMatrix(data=X1, label=y1)

In [22]:
param = {'max_depth': 10, 'eta': 0.1, 'objective': 'reg:squarederror'}
evals = [(dtrain, 'train'), (dvali, 'validacion')]

Comprobación de la tasa de error en los datos de test

In [24]:
def evalXGB(Test_xgb, predict_xgb_test):
    suma = 0
    n = len(Test_xgb)
    for i in range(0,n):
        suma = abs(predict_xgb_test[i] - Test_xgb[i])/Test_xgb[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [32]:
def train_XGB(num_boost_round, early_stop, maxdepthmin, maxdepthmax):
    best = 100
    best_depth = 0
    for i in range(maxdepthmin, maxdepthmax):
        param = {'max_depth': i, 'eta': 0.1, 'objective': 'reg:squarederror'}
        bstaux = xgb.train(param, dtrain, num_boost_round=num_boost_round, evals=evals, early_stopping_rounds=early_stop, verbose_eval=10)
        predict_xgb_test = bstaux.predict(dtrainvali)
        valor = evalXGB(yvtXGB, predict_xgb_test)
        if(valor < best):
            best = valor
            best_depth = i
    return (best_depth, best)

In [ ]:
train_XGB(1000, 10, 1, 15)

[0]	train-rmse:53.26953	validacion-rmse:55.23855
[10]	train-rmse:26.63411	validacion-rmse:21.66486
[20]	train-rmse:15.80944	validacion-rmse:9.74252
[30]	train-rmse:10.10769	validacion-rmse:6.02822
[40]	train-rmse:6.72877	validacion-rmse:4.32861
[50]	train-rmse:4.62899	validacion-rmse:3.40639
[60]	train-rmse:3.34697	validacion-rmse:2.84340
[70]	train-rmse:2.60039	validacion-rmse:2.59542
[80]	train-rmse:2.18529	validacion-rmse:2.50218
[90]	train-rmse:1.96376	validacion-rmse:2.44816
[100]	train-rmse:1.84537	validacion-rmse:2.42617
[110]	train-rmse:1.78430	validacion-rmse:2.41209
[120]	train-rmse:1.75328	validacion-rmse:2.41156
[0]	train-rmse:52.39037	validacion-rmse:55.17754
[10]	train-rmse:20.23810	validacion-rmse:19.20769
[20]	train-rmse:8.13854	validacion-rmse:7.27882
[30]	train-rmse:3.55046	validacion-rmse:3.21334
[40]	train-rmse:1.94405	validacion-rmse:2.06142
[50]	train-rmse:1.48697	validacion-rmse:1.81909
[60]	train-rmse:1.38378	validacion-rmse:1.78013
[70]	train-rmse:1.35802	valid

(8, 0.9708896746706613)